# Rekognition による物体検出

## 必要なパッケージのインポート

In [ ]:
import io
import requests
import boto3
from PIL import Image, ImageDraw

## アカウント情報の確認

In [ ]:
boto3.client('sts').get_caller_identity()

## クライアントの作成

In [ ]:
client = boto3.client('rekognition')

## ローカル画像からの検出

In [ ]:
img_pil = Image.open('imgs/sofa.jpg')
img_pil

In [ ]:
img_bytes_io = io.BytesIO()
img_pil.save(img_bytes_io, format="jpeg")
img_bytes = img_bytes_io.getvalue()

response = client.detect_labels(
    Image={
        'Bytes': img_bytes
    },
    MaxLabels=5,
    MinConfidence=0.5
)

response

In [ ]:
def show_labels(img, labels):
    img_width, img_height = img.size
    draw = ImageDraw.Draw(img)

    for label in labels:
        for instance in label['Instances']:
            box = instance['BoundingBox']
            draw.rectangle(
                (box['Left'] * img_width, box['Top'] * img_height, # 左上座標
                (box['Left'] + box['Width']) * img_width, (box['Top'] + box['Height']) * img_height), # 右下座標
                outline=(255, 0, 0) # 色
            )
            draw.text(
                (box['Left'] * img_width + 5, box['Top'] * img_height + 5), # 左上座標
                label['Name'], # 文字
                fill=(255, 0, 0) # 色
            )
            draw.text(
                (box['Left'] * img_width + 5, box['Top'] * img_height + 18), # 左上座標
                str(round(instance['Confidence'], 1)) + ' %', # 文字
                fill=(255, 0, 0) # 色
            )

    return img

In [ ]:
show_labels(img_pil.copy(), response['Labels'])

## Web上の画像からの検出

In [ ]:
img_web_res = requests.get('http://free-photo.net/photo_img/0812110532.jpg')
img_web = Image.open(io.BytesIO(img_web_res.content))
img_web

In [ ]:
img_bytes = io.BytesIO(img_web_res.content).getvalue()

response = client.detect_labels(
    Image={
        'Bytes': img_bytes
    },
    MaxLabels=5,
    MinConfidence=0.5
)

response

In [ ]:
show_labels(img_web.copy(), response['Labels'])